# 충돌이 너무 많아서 사용불가

# # Git 클론
---

In [ ]:
# 깃에서 가져오기

%cd /content/drive/MyDrive/workspace/study/3D_Object_detection
!git clone https://github.com/hailanyi/3D-Detection-Tracking-Viewer

/content/drive/MyDrive/workspace/study/3D_Object_detection
Cloning into '3D-Detection-Tracking-Viewer'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 203 (delta 104), reused 119 (delta 43), pack-reused 0
Receiving objects: 100% (203/203), 24.65 MiB | 17.28 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [ ]:
# 모듈 인스톨

%cd /content/drive/MyDrive/workspace/study/3D_Object_detection/3D-Detection-Tracking-Viewer
!pip install -r requirements.txt

/content/drive/MyDrive/workspace/study/3D_Object_detection/3D-Detection-Tracking-Viewer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 3.6 MB/s 
     |████████████████████████████████| 9.3 MB 73.1 MB/s 
     |████████████████████████████████| 59.5 MB 1.1 MB/s 
     |████████████████████████████████| 60.3 MB 1.4 MB/s 
     |████████████████████████████████| 10.3 MB 45.9 MB/s 
     |████████████████████████████████| 12.0 MB 65.1 MB/s 
     |████████████████████████████████| 3.1 MB 84.6 MB/s 
     |████████████████████████████████| 476 kB 96.8 MB/s 
     |████████████████████████████████| 271 kB 89.4 MB/s 
     |████████████████████████████████| 4.0 MB 79.4 MB/s 
     |████████████████████████████████| 74 kB 3.7 MB/s 
     |████████████████████████████████| 1.6 MB 78.9 MB/s 
     |████████████████████████████████| 251 kB 85.1 MB/s 
  Created wheel for vedo: filename=vedo-2021.0.6-py3-none-a

# # 사용법
---

## # kitti PCD bin to txt
---

In [ ]:
import struct
import sys
from struct import unpack
def readBinFile(fname):
    x = []
    y = []
    z = []
    try:
        with open(fname, 'rb') as fp:
            while True:
                val1 = unpack('<1f', fp.read(4))
                val2 = unpack('<1f', fp.read(4))
                val3 = unpack('<1f', fp.read(4))
                val4 = unpack('<1f', fp.read(4))
                x.append(val1[0])
                y.append(val2[0])
                z.append(val3[0])
                print(val1[0], val2[0], val3[0])
    except Exception as e:
        print(e)
        pass
    finally:
        fp.close()  # 파일 닫기
    return x,y,z

def writePCDFile(fname,x,y,z):
    numPoints= len(x)
    with open(fname, 'w') as fp:
        fp.write("VERSION 0.7\n")
        fp.write("FIELDS x y z\n")
        fp.write("SIZE 4 4 4\n")
        fp.write("TYPE F F F\n")
        fp.write("WIDTH "+str(numPoints)+"\n")
        fp.write("HEIGHT 1\n")
        fp.write("POINTS "+str(numPoints)+"\n")
        fp.write("DATA ascii\n")
        for index in range(numPoints):
            txtLine = "{} {} {}\n".format(x[index],y[index],z[index] )
            fp.write(txtLine)
        pass

# if __name__ == "__main__":
#     fname1 ="../StereoDataset/000001.bin"
#     fname2 ="../StereoDataset/000001.pcd"
#     x,y,z =readBinFile(fname1)
#     writePCDFile(fname2, x, y, z)
#     pass

## # 상자 유형 및 뷰어 배경색 설정
---
현재 이 코드는 Kitti(h,w,l,x,y,z,yaw) 및 Waymo OpenPCDet(x,y,z,l,w,h,yaw) 상자 유형을 지원합니다. 뷰어 초기화 시 상자의 종류와 배경색을 다음과 같이 설정할 수 있습니다.

In [ ]:
%cd /content/drive/MyDrive/workspace/study/3D_Object_detection/3D-Detection-Tracking-Viewer

/content/drive/MyDrive/workspace/study/3D_Object_detection/3D-Detection-Tracking-Viewer


In [ ]:
from viewer.viewer import Viewer

vi = Viewer(box_type="Kitti",bg = (255,255,255))

## # 개체 색상 맵 설정
---
matplotlab.pypot 컬러 맵 과 동일하게 뷰 추적 결과에 대한 객체 컬러 맵을 설정할 수 있습니다 . 일반적으로 사용되는 색상 맵은 "rainbow", "viridis", "brg", "gnuplot", "hsv" 등입니다.

In [ ]:
vi.set_ob_color_map('rainbow')

## # 3D 장면에 컬러 포인트 클라우드 추가
---
뷰어는 점 세트를 수신하며 모양이 (N,3)인 배열이어야 합니다. 스캐터 필드를 보려면 'scatter_filed'를 모양(N,)으로 설정하고 'color_map_name'을 설정하여 색상을 지정해야 합니다. 'scatter_filed'가 None이면 포인트는 'color' 인수의 색상으로 표시됩니다.

In [ ]:
vi.add_points(points_array.T,
               radius = 2,
               color = (150,150,150),
               scatter_filed = None,
               alpha=1,
               del_after_show = True,
               add_to_3D_scene = True,
               add_to_2D_scene = True,
               color_map_name = "viridis")

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
from viewer.viewer import Viewer
import numpy as np
from dataset.kitti_dataset import KittiTrackingDataset

def kitti_viewer():
    root="/content/drive/MyDrive/workspace/study/3D_Object_detection/OpenPCDet/data/kittimot/training"
    # label_path = "/content/drive/MyDrive/workspace/study/3D_Object_detection/OpenPCDet/data/kittimot/training/label_02"
    # dataset = KittiTrackingDataset(root,seq_id=1,label_path=label_path)
    dataset = KittiTrackingDataset(root,seq_id=1)


    vi = Viewer(box_type="Kitti")

    for i in range(len(dataset)):
        P2, V2C, points, image, labels, label_names = dataset[i]


        if labels is not None:
            mask = (label_names=="Car")
            labels = labels[mask]
            label_names = label_names[mask]
            vi.add_3D_boxes(labels, ids=labels[:, -1].astype(int), box_info=label_names,caption_size=(0.09,0.09))
            vi.add_3D_cars(labels, ids=labels[:, -1].astype(int), mesh_alpha=1)
        vi.add_points(points[:,:3])

        vi.add_image(image)
        vi.set_extrinsic_mat(V2C)
        vi.set_intrinsic_mat(P2)

        vi.show_2D()
        vi.show_3D()

In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/MyDrive/workspace/study/3D_Object_detection/3D-Detection-Tracking-Viewer


from viewer.viewer import Viewer
import numpy as np
from dataset.kitti_dataset import KittiTrackingDataset
from google.colab.patches import cv2_imshow

root="/content/drive/MyDrive/workspace/study/3D_Object_detection/OpenPCDet/data/kittimot/training"
# label_path = "/content/drive/MyDrive/workspace/study/3D_Object_detection/OpenPCDet/data/kittimot/training/label_02"
# dataset = KittiTrackingDataset(root,seq_id=1,label_path=label_path)
dataset = KittiTrackingDataset(root,seq_id=1)


vi = Viewer(box_type="Kitti")

P2, V2C, points, image, labels, label_names = dataset[1]


if labels is not None:
    mask = (label_names=="Car")
    labels = labels[mask]
    label_names = label_names[mask]
    vi.add_3D_boxes(labels, ids=labels[:, -1].astype(int), box_info=label_names,caption_size=(0.09,0.09))
    vi.add_3D_cars(labels, ids=labels[:, -1].astype(int), mesh_alpha=1)
vi.add_points(points[:,:3])

vi.add_image(image)
vi.set_extrinsic_mat(V2C)
vi.set_intrinsic_mat(P2)

vi.show_2D()



/content/drive/MyDrive/workspace/study/3D_Object_detection/3D-Detection-Tracking-Viewer
embedWindow(verbose=True): could not load ipyvtklink try:
> pip install ipyvtklink


ModuleNotFoundError: ignored

In [ ]:
!pip install k3d


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.1 MB 4.1 MB/s 


In [ ]:
from vedo import dataurl, settings, Volume, Plotter

settings.useDepthPeeling = True

vol = Volume(dataurl+'embryo.tif') # load Volume
vol.print()

# show lego blocks whose value is between vmin and vmax
lego = vol.legosurface(vmin=65).cmap('seismic').addScalarBar()

plt = Plotter(backend='ipygany', axes=1, bg='wheat', bg2='lb', size=(900,600))
plt.show(lego, viewup='z', zoom=1.75)

_________________________________________________________________
Volume
         position: (0.0, 0.0, 0.0)
       dimensions: (125, 80, 107)
          spacing: (104.03866670068368, 104.03866670068368, 104.03866670068368)
   data dimension: 3
      memory size: 1 MB
    scalar #bytes: 1
           bounds: x=(0, 1.29e+4) y=(0, 8.22e+3) z=(0, 1.10e+4)
     scalar range: (0.0, 150.0)
(logscale) Histogram	(entries=100000)
                                      4.91
0.00  | ▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
18.38 | ▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
36.75 | ▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
55.12 | ▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
73.50 | ▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
91.88 | ▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
110.25| ▆▆▆▆▆▆▆▆▆▆▆▆▆
128.62| ▆▆▆▆▆▆▆▆

In cmap(): cannot find any active Point array ...skip coloring.


TypeError: ignored

In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autobahn                      22.7.1
autograd                      1.4
Automat                       20.2.0
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()